<a href="https://colab.research.google.com/github/nizar123456789/ReinforcementLearning_Algorithms/blob/main/Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import gym 

In [39]:
# HIDE OUTPUT
!wget http://www.atarimania.com/roms/Roms.rar 
!unrar x -o+ /content/Roms.rar >/dev/nul
!python -m atari_py.import_roms /content/ROMS >/dev/nul

--2023-02-10 16:31:34--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19612325 (19M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.4’

Roms.rar.4          100%[===================>]  18.70M   705KB/s    in 28s     

2023-02-10 16:32:02 (691 KB/s) - ‘Roms.rar.4’ saved [19612325/19612325]



In [ ]:
# HIDE OUTPUT
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

In [ ]:
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""


def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    env = RecordVideo(env, './video')
    return env


In [ ]:
!pip install gym[classic_control]

In [ ]:


env = wrap_env(gym.make("MountainCar-v0"))
#env = wrap_env(gym.make("Atlantis-v0"))
env.reset()
done=False
while not done:
    action=2
    env.render()
    new_state,reward,done,_=env.step(action)
    print(new_state)


    # your agent goes here
    #action = env.action_space.sample()

    #observation, reward, done, info = env.step(action)

    #if done:
        #break

env.close()
show_video()


In [ ]:
import numpy as np
env = wrap_env(gym.make("MountainCar-v0"))

DESCRETE_OS_SIZE=[20]*len(env.observation_space.high)
discrete_os_wind_size=(env.observation_space.high-env.observation_space.low)/20
q_table=np.random.uniform(low=-2,high=0,size=(DESCRETE_OS_SIZE+[env.action_space.n]))
#the q table contains every combination of observation with position and vilocity
print(q_table.shape)
print(q_table)

In [ ]:
def get_discrete_state(state):
  discrete_state=(state - env.observation_space.low)/discrete_os_wind_size
  return tuple(discrete_state.astype(np.int))

discrete_state=get_discrete_state(env.reset())
print(discrete_state)

In [ ]:
EPISODES=25000
SHOW_EVERY_THING=2000
LEARNING_RATE=0.1
DISCOUNT_RATE=0.95
EPSILONE=0.5
START_EPSILONE_DECAYING=1
END_EPSILONE_DECAYING=EPISODES//2
epsilone_decay_value=EPSILONE/(START_EPSILONE_DECAYING-END_EPSILONE_DECAYING)

In [37]:


for episode in range(EPISODES):
  if episode %SHOW_EVERY_THING ==0:
    render=True
  else:
    render=False
  
  discrete_state=get_discrete_state(env.reset())

  done=False
  
  while not done:
    action=np.argmax(q_table[discrete_state])
    new_state,reward,done,_=env.step(action)
    new_discrete_state=get_discrete_state(new_state)
    #if render:
      #env.render()
   
   
    if not done:
       max_future_q=np.max(q_table[new_discrete_state])
       current_q=q_table[new_discrete_state+(action, )]
       new_q=(1-LEARNING_RATE)*current_q+LEARNING_RATE*(reward+(1-DISCOUNT_RATE)*max_future_q)
       current_q=new_q
    elif new_state[0]>=env.goal_position:
      print(f"we made it on episode {episode}")
      q_table[discrete_state+(action, )]=0
      discrete_state=new_discrete_state
    if END_EPSILONE_DECAYING>=episode>=START_EPSILONE_DECAYING:
      EPSILONE-=epsilone_decay_value

env.close()
#show_video()




<ipython-input-32-13999ffa34ba>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(discrete_state.astype(np.int))
/usr/local/lib/python3.8/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment MountainCar-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode dur

KeyboardInterrupt: ignored